In [8]:
import pre_process_data
import Custom_dataset
import Transformer_base_model
import check_accuracy_and_save_model
#import training
import TrainHelper
import graphsAndPlots
from torch import optim
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch
import PretraindTransformerModel
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import os
import PretraindModelCustomDataset

import seaborn as sns
seed = 0
torch.manual_seed(seed)
#torch.cuda.manual_seed_all(seed)

import random
random.seed(0)
np.random.seed(0)

from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
import optuna

import tqdm
from tqdm import tqdm
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score
import numpy as np

import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.lines import Line2D

In [9]:
def marginal_loss(embeddings, labels, margin=1.0):
    # Calculate pairwise distance matrix
    distance_matrix = torch.cdist(embeddings, embeddings, p=2)

    # Transpose the labels tensor from [16, 1] to [1, 16]
    labels_t = labels.transpose(0, 1)

    # Create masks for pairs of the same and different classes
    same_class_mask = torch.eq(labels, labels_t).float()
    different_class_mask = 1 - same_class_mask

    # Calculate loss for pairs of the same class (should be small)
    same_class_loss = torch.mean(F.relu(distance_matrix) * same_class_mask)

    # Calculate loss for pairs of different classes (should be larger than margin)
    different_class_loss = torch.mean(F.relu(margin - distance_matrix) * different_class_mask)

    # Combine the two components of the loss
    loss = same_class_loss + different_class_loss

    return loss

def weighted_marginal_loss(embeddings, labels, margin=1.0, class_weight=None):
    """
    embeddings: Tensor - The embeddings from the model
    labels: Tensor - Corresponding labels for the embeddings
    margin: float - The margin value for different class pairs
    class_weight: dict - Weights for each class {class_label: weight}
    """
    # Calculate pairwise distance matrix
    distance_matrix = torch.cdist(embeddings, embeddings, p=2)

    # Expand labels to match dimensions
    labels = labels.unsqueeze(1)
    labels_t = labels.transpose(0, 1)

    # Create masks for pairs of same and different classes
    same_class_mask = torch.eq(labels, labels_t).float()
    different_class_mask = 1 - same_class_mask

    # Apply class weights if provided
    if class_weight is not None:
        weights = labels.squeeze().apply_(lambda x: class_weight.get(x.item(), 1.0))
        weight_matrix = weights.unsqueeze(1) * weights.unsqueeze(0)
    else:
        weight_matrix = torch.ones_like(same_class_mask)

    # Calculate losses
    same_class_loss = torch.mean(F.relu(distance_matrix) * same_class_mask * weight_matrix)
    different_class_loss = torch.mean(F.relu(margin - distance_matrix) * different_class_mask * weight_matrix)

    return same_class_loss + different_class_loss

In [10]:
params = {
        'embed_size':60,
        'num_layers': 1,
        'heads': 6,
        'forward_expansion': 6,
        'dropout': 0.35419923563696654,
        'l_r': 0.003,
        'batch': 16
    }

other_data_pairs = [
    ('Cas9-Group1Seq.xlsx', 'Cas12.xlsx'),
    #('Cas12.xlsx', 'nonCas_labels_0.xlsx'),
    #('Cas9-Group1Seq.xlsx', 'nonCas_labels_1.xlsx')  # Add more pairs as needed
    #('Cas9-Group1Seq_vs_Cas12_merged_data.xlsx', 'nonCas_labels_2.xlsx')
]
k_mer = 1
k_folds = 10
#num_runs = 10  # Specify the number of runs
latent_space = []
labels_space = []
#attention_matrices = {class_1: [], class_2: []}
for other_data_pair in other_data_pairs:
    #output_dir = "_vs_".join([file_name.split(".")[0] for file_name in other_data_pair])+"_Cross_Validation_Results"
    output_file_name = "_vs_".join([file_name.split(".")[0] for file_name in other_data_pair]) + "_Final_Data.csv"
    data = pd.read_csv(output_file_name)
    with open(output_file_name.replace(".csv", "_Variables.pkl"), 'rb') as f:
        word2index, index2word, source_vocab_max_size, vocab = pickle.load(f)

    base_folder = r'F:\Back Up\Desktop\Research Project Classification Files\Small Data -Original Data\Final Filder Classification task\Cas Classification'

# List of different folder names you want to process
folder_names = [
    'Cas9-Group1Seq_vs_Cas12_Cross_Validation_Results',
    
    # Add more folder names as needed
]
vocab_size = len(word2index)
num_folds = 10  # Update this based on the number of folds you have
max_length = source_vocab_max_size  # Max length of any sequence
epochs = 40
device = "cuda"
patience = 5

latent_space = []
labels_space = []
for folder_name in folder_names:
    for fold in range(1, num_folds + 1):
        # Dynamic paths for checkpoint and data files
        
        fold_dir = os.path.join(base_folder, folder_name, f"Fold_{fold}")
        os.makedirs(fold_dir, exist_ok=True)
        
        checkpoint_path = os.path.join(base_folder, f'{folder_name}\\bestmodel_kmer_1_fold_{fold}.pth')
        data_folder = os.path.join(base_folder, f'{folder_name}\\kmer_1_fold_{fold}')
        
        train_csv_path = os.path.join(data_folder, 'train.csv')
        test_csv_path = os.path.join(data_folder, 'test.csv')
        
        # Load the data
        train_data = pd.read_csv(train_csv_path)
        test_data = pd.read_csv(test_csv_path)
        
        # Create datasets
        train_dataset = Custom_dataset.MyDataset(train_csv_path, source_vocab_max_size, len(word2index))
        test_dataset = Custom_dataset.MyDataset(test_csv_path, source_vocab_max_size, len(word2index))
        
        # DataLoader setup
        train_loader = DataLoader(train_dataset, batch_size=params['batch'], shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=params['batch'], shuffle=False)

        # Model setup
        model = Transformer_base_model.Encoder(vocab_size, params['embed_size'], params['num_layers'], params['heads'], device, params['forward_expansion'], params['dropout'], max_length, params['batch']).to(device)
        
        # Load model checkpoint
        checkpoint = torch.load(checkpoint_path)
        model.load_state_dict(checkpoint['model_state_dict'], strict=False)
        
        # Loss function and optimizer setup
        criterion_bce = nn.BCEWithLogitsLoss()
        optimizer = optim.Adam(model.parameters(), lr=params['l_r'])

        best_confusion_matrix = None
        best_model = None
        highest_accuracy = 0.0
        training_losses = []
        test_accuracies = []
        best_epoch = 0
        # Training loop for the current fold
        for epoch in range(epochs):
            model.train()
            running_loss_bce = 0.0
            running_loss_marginal = 0.0
            total_loss = 0.0
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()

                output, embeddings = model(inputs)
                loss_bce = criterion_bce(output, labels)
                loss_marginal = marginal_loss(embeddings, labels, margin=20)

                # Combine the losses
                combined_loss = loss_bce + loss_marginal
                combined_loss.backward()
                optimizer.step()
                running_loss_bce += loss_bce.item()
                running_loss_marginal += loss_marginal.item()
                total_loss += combined_loss.item()
                
            avg_loss = total_loss / len(train_loader)
            training_losses.append(avg_loss)
            # Print average losses after each epoch
            avg_loss_bce = running_loss_bce / len(train_loader)
            avg_loss_marginal = running_loss_marginal / len(train_loader)
            print(f"Epoch [{epoch+1}/{epochs}] - Fold {fold} - Avg BCE Loss: {avg_loss_bce:.4f}, Avg Marginal Loss: {avg_loss_marginal:.4f}")
                
            # Evaluation
            model.eval()
            accuracy, avg_loss, confusion_matrix = check_accuracy_and_save_model.check_accuracy(test_loader, model, criterion_bce)
            
            # Check for the best model
            if accuracy > highest_accuracy:
                highest_accuracy = accuracy
                best_model = model.state_dict()
                best_confusion_matrix = confusion_matrix
                best_epoch = epoch
                print(f"Fold {fold}: Accuracy: {accuracy}%, Avg Loss: {avg_loss}")
                print(f"Confusion Matrix:\n{confusion_matrix}")
                
            fpr, tpr, thresholds = check_accuracy_and_save_model.ROC(test_loader, model, criterion_bce)
            roc_auc = auc(fpr, tpr)
            print(f"Fold {fold}: ROC AUC: {roc_auc}")

            torch.cuda.empty_cache()
        # Save the best model
        best_model_path = os.path.join(fold_dir, 'best_model_marignal_loss_finetune.pth')
        torch.save(best_model, best_model_path)

        # Plot and save the training loss graph
        plt.figure()
        plt.plot(training_losses, label='Training Loss')
        plt.axvline(x=best_epoch, color='grey', linestyle='--', label=f'Best Epoch: {best_epoch+1}')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.title(f'Training Loss for Fold {fold}')
        plt.legend()
        plt.savefig(os.path.join(fold_dir, 'training_loss.png'))
        plt.close()
        
        TN, FP, FN, TP = best_confusion_matrix.ravel()

        accuracy = (TP + TN) / (TP + TN + FP + FN)
        precision = TP / (TP + FP) if (TP + FP) != 0 else 0
        recall = TP / (TP + FN) if (TP + FN) != 0 else 0
        f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) != 0 else 0
        specificity = TN / (TN + FP) if (TN + FP) != 0 else 0
        balanced_accuracy = (recall + specificity) / 2

        print(f"Fold {fold}, Best Epoch {best_epoch+1}:")
        print(f"Accuracy: {accuracy:.8f}, Precision: {precision:.8f}, Recall: {recall:.8f}, F1 Score: {f1_score:.8f}, Specificity: {specificity:.2f}, Balanced Accuracy: {balanced_accuracy:.8f}")
        print()
        print()
        print()

Epoch [1/40] - Fold 1 - Avg BCE Loss: 0.0555, Avg Marginal Loss: 4.7497
Fold 1: Accuracy: 90.26548672566372%, Avg Loss: 0.6351510022259319
Confusion Matrix:
[[84  0]
 [11 18]]
Fold 1: ROC AUC: 0.9819376026272578
Epoch [2/40] - Fold 1 - Avg BCE Loss: 0.1075, Avg Marginal Loss: 2.1184
Fold 1: Accuracy: 96.46017699115045%, Avg Loss: 0.11546095954327029
Confusion Matrix:
[[81  3]
 [ 1 28]]
Fold 1: ROC AUC: 0.9958949096880131
Epoch [3/40] - Fold 1 - Avg BCE Loss: 0.0845, Avg Marginal Loss: 1.6126
Fold 1: ROC AUC: 0.9651067323481117
Epoch [4/40] - Fold 1 - Avg BCE Loss: 0.0765, Avg Marginal Loss: 1.3232
Fold 1: ROC AUC: 0.9655172413793104
Epoch [5/40] - Fold 1 - Avg BCE Loss: 0.0620, Avg Marginal Loss: 1.3045
Fold 1: ROC AUC: 0.992200328407225
Epoch [6/40] - Fold 1 - Avg BCE Loss: 0.0616, Avg Marginal Loss: 1.3135
Fold 1: ROC AUC: 0.9671592775041051
Epoch [7/40] - Fold 1 - Avg BCE Loss: 0.0774, Avg Marginal Loss: 1.3083
Fold 1: Accuracy: 97.34513274336283%, Avg Loss: 0.07678376327385195
Conf

Fold 2: ROC AUC: 0.9991789819376027
Epoch [26/40] - Fold 2 - Avg BCE Loss: 0.0035, Avg Marginal Loss: 0.4682
Fold 2: ROC AUC: 0.9995894909688013
Epoch [27/40] - Fold 2 - Avg BCE Loss: 0.0029, Avg Marginal Loss: 0.4988
Fold 2: ROC AUC: 0.965927750410509
Epoch [28/40] - Fold 2 - Avg BCE Loss: 0.0095, Avg Marginal Loss: 0.5839
Fold 2: ROC AUC: 0.9991789819376026
Epoch [29/40] - Fold 2 - Avg BCE Loss: 0.0036, Avg Marginal Loss: 0.4096
Fold 2: ROC AUC: 0.9995894909688013
Epoch [30/40] - Fold 2 - Avg BCE Loss: 0.0038, Avg Marginal Loss: 0.4247
Fold 2: ROC AUC: 0.9995894909688013
Epoch [31/40] - Fold 2 - Avg BCE Loss: 0.0063, Avg Marginal Loss: 0.4057
Fold 2: ROC AUC: 0.9995894909688013
Epoch [32/40] - Fold 2 - Avg BCE Loss: 0.0111, Avg Marginal Loss: 0.4403
Fold 2: ROC AUC: 0.9995894909688013
Epoch [33/40] - Fold 2 - Avg BCE Loss: 0.0017, Avg Marginal Loss: 0.3478
Fold 2: ROC AUC: 0.9995894909688013
Epoch [34/40] - Fold 2 - Avg BCE Loss: 0.0048, Avg Marginal Loss: 0.3900
Fold 2: ROC AUC: 0.9

Fold 4: ROC AUC: 0.9995894909688013
Epoch [15/40] - Fold 4 - Avg BCE Loss: 0.0275, Avg Marginal Loss: 0.7558
Fold 4: ROC AUC: 1.0
Epoch [16/40] - Fold 4 - Avg BCE Loss: 0.0310, Avg Marginal Loss: 0.8069
Fold 4: ROC AUC: 0.9991789819376026
Epoch [17/40] - Fold 4 - Avg BCE Loss: 0.0237, Avg Marginal Loss: 0.6972
Fold 4: ROC AUC: 0.9991789819376026
Epoch [18/40] - Fold 4 - Avg BCE Loss: 0.0253, Avg Marginal Loss: 0.7131
Fold 4: ROC AUC: 1.0
Epoch [19/40] - Fold 4 - Avg BCE Loss: 0.0156, Avg Marginal Loss: 0.6619
Fold 4: ROC AUC: 0.9995894909688013
Epoch [20/40] - Fold 4 - Avg BCE Loss: 0.0179, Avg Marginal Loss: 0.6488
Fold 4: ROC AUC: 0.9991789819376026
Epoch [21/40] - Fold 4 - Avg BCE Loss: 0.0179, Avg Marginal Loss: 0.6394
Fold 4: ROC AUC: 1.0
Epoch [22/40] - Fold 4 - Avg BCE Loss: 0.0084, Avg Marginal Loss: 0.5281
Fold 4: ROC AUC: 1.0
Epoch [23/40] - Fold 4 - Avg BCE Loss: 0.0151, Avg Marginal Loss: 0.6014
Fold 4: ROC AUC: 0.9991789819376026
Epoch [24/40] - Fold 4 - Avg BCE Loss: 0.01

Fold 6: ROC AUC: 0.9982993197278912
Epoch [2/40] - Fold 6 - Avg BCE Loss: 0.0898, Avg Marginal Loss: 2.1776
Fold 6: ROC AUC: 0.9982993197278912
Epoch [3/40] - Fold 6 - Avg BCE Loss: 0.0950, Avg Marginal Loss: 1.6562
Fold 6: ROC AUC: 0.9982993197278911
Epoch [4/40] - Fold 6 - Avg BCE Loss: 0.0665, Avg Marginal Loss: 1.4104
Fold 6: ROC AUC: 0.9987244897959183
Epoch [5/40] - Fold 6 - Avg BCE Loss: 0.0805, Avg Marginal Loss: 1.3753
Fold 6: ROC AUC: 0.9987244897959183
Epoch [6/40] - Fold 6 - Avg BCE Loss: 0.0690, Avg Marginal Loss: 1.2435
Fold 6: ROC AUC: 0.9991496598639457
Epoch [7/40] - Fold 6 - Avg BCE Loss: 0.0650, Avg Marginal Loss: 1.1853
Fold 6: ROC AUC: 0.9995748299319729
Epoch [8/40] - Fold 6 - Avg BCE Loss: 0.0600, Avg Marginal Loss: 1.1708
Fold 6: ROC AUC: 0.9987244897959183
Epoch [9/40] - Fold 6 - Avg BCE Loss: 0.0601, Avg Marginal Loss: 1.1463
Fold 6: ROC AUC: 0.9991496598639455
Epoch [10/40] - Fold 6 - Avg BCE Loss: 0.0617, Avg Marginal Loss: 1.1837
Fold 6: ROC AUC: 0.99914965

Fold 7: ROC AUC: 0.9642857142857144
Epoch [33/40] - Fold 7 - Avg BCE Loss: 0.0029, Avg Marginal Loss: 0.3699
Fold 7: ROC AUC: 0.9311224489795918
Epoch [34/40] - Fold 7 - Avg BCE Loss: 0.0091, Avg Marginal Loss: 0.4706
Fold 7: ROC AUC: 0.931547619047619
Epoch [35/40] - Fold 7 - Avg BCE Loss: 0.0068, Avg Marginal Loss: 0.4173
Fold 7: ROC AUC: 0.9647108843537416
Epoch [36/40] - Fold 7 - Avg BCE Loss: 0.0020, Avg Marginal Loss: 0.3228
Fold 7: ROC AUC: 0.9298469387755102
Epoch [37/40] - Fold 7 - Avg BCE Loss: 0.0075, Avg Marginal Loss: 0.3885
Fold 7: ROC AUC: 0.9647108843537414
Epoch [38/40] - Fold 7 - Avg BCE Loss: 0.0034, Avg Marginal Loss: 0.3638
Fold 7: ROC AUC: 0.9642857142857143
Epoch [39/40] - Fold 7 - Avg BCE Loss: 0.0007, Avg Marginal Loss: 0.3253
Fold 7: ROC AUC: 0.9396258503401361
Epoch [40/40] - Fold 7 - Avg BCE Loss: 0.0027, Avg Marginal Loss: 0.3533
Fold 7: ROC AUC: 0.929421768707483
Fold 7, Best Epoch 4:
Accuracy: 0.98214286, Precision: 1.00000000, Recall: 0.92857143, F1 Scor

Fold 9: ROC AUC: 0.9418363107602825
Epoch [20/40] - Fold 9 - Avg BCE Loss: 0.0107, Avg Marginal Loss: 0.4719
Fold 9: ROC AUC: 0.9356044869131699
Epoch [21/40] - Fold 9 - Avg BCE Loss: 0.0049, Avg Marginal Loss: 0.4844
Fold 9: ROC AUC: 0.9019526381387619
Epoch [22/40] - Fold 9 - Avg BCE Loss: 0.0034, Avg Marginal Loss: 0.4337
Fold 9: ROC AUC: 0.9011217282924803
Epoch [23/40] - Fold 9 - Avg BCE Loss: 0.0055, Avg Marginal Loss: 0.4162
Fold 9: ROC AUC: 0.9007062733693395
Epoch [24/40] - Fold 9 - Avg BCE Loss: 0.0028, Avg Marginal Loss: 0.3399
Fold 9: ROC AUC: 0.8998753635230577
Epoch [25/40] - Fold 9 - Avg BCE Loss: 0.0077, Avg Marginal Loss: 0.4421
Fold 9: ROC AUC: 0.899459908599917
Epoch [26/40] - Fold 9 - Avg BCE Loss: 0.0105, Avg Marginal Loss: 0.4953
Fold 9: ROC AUC: 0.9966763606148732
Epoch [27/40] - Fold 9 - Avg BCE Loss: 0.0037, Avg Marginal Loss: 0.4177
Fold 9: ROC AUC: 0.9007062733693395
Epoch [28/40] - Fold 9 - Avg BCE Loss: 0.0006, Avg Marginal Loss: 0.3387
Fold 9: ROC AUC: 0.9

In [11]:
counter = 0
params = {
        'embed_size':60,
        'num_layers': 2,
        'heads': 6,
        'forward_expansion': 2,
        'dropout': 0.35419923563696654,
        'l_r': 0.003,
        'batch': 1
    }

other_data_pairs = [
    #('Cas9-Group1Seq.xlsx', 'Cas12.xlsx'),
    ('Cas12.xlsx', 'nonCas_labels_0.xlsx'),
    #('Cas9-Group1Seq.xlsx', 'nonCas_labels_1.xlsx')  # Add more pairs as needed
    #('Cas9-Group1Seq_vs_Cas12_merged_data.xlsx', 'nonCas_labels_2.xlsx')
]
k_mer = 1
k_folds = 10
#num_runs = 10  # Specify the number of runs
latent_space = []
labels_space = []
#attention_matrices = {class_1: [], class_2: []}
for other_data_pair in other_data_pairs:
    #output_dir = "_vs_".join([file_name.split(".")[0] for file_name in other_data_pair])+"_Cross_Validation_Results"
    output_file_name = "_vs_".join([file_name.split(".")[0] for file_name in other_data_pair]) + "_Final_Data.csv"
    data = pd.read_csv(output_file_name)
    with open(output_file_name.replace(".csv", "_Variables.pkl"), 'rb') as f:
        word2index, index2word, source_vocab_max_size, vocab = pickle.load(f)

    base_folder = r'F:\Back Up\Desktop\Research Project Classification Files\Small Data -Original Data\Final Filder Classification task\Cas Classification'

# List of different folder names you want to process
folder_names = [
    'Cas12_vs_nonCas_labels_0_Cross_Validation_Results'
    # Add more folder names as needed
]
vocab_size = len(word2index)
num_folds = 10  # Update this based on the number of folds you have
max_length = source_vocab_max_size  # Max length of any sequence
epochs = 40
device = "cuda"
patience = 5

latent_space = []
for folder_name in folder_names:
    for fold in range(1, num_folds + 1):
        print(f"Processing: {folder_name}, Fold: {fold}")

        # Paths for checkpoint and data files
        checkpoint_path = os.path.join(base_folder, f'{folder_name}\\Fold_{fold}\\best_model_marignal_loss_finetune.pth')
        train_csv_path = os.path.join(base_folder, f'{folder_name}\\kmer_1_fold_{fold}', 'train.csv')
        test_csv_path = os.path.join(base_folder, f'{folder_name}\\kmer_1_fold_{fold}', 'test.csv')

        # Load data
        train = Custom_dataset.MyDataset(train_csv_path, source_vocab_max_size, len(word2index))
        test = Custom_dataset.MyDataset(test_csv_path, source_vocab_max_size, len(word2index))
        train_loader = DataLoader(train, batch_size=params['batch'], shuffle=True)
        test_loader = DataLoader(test, batch_size=params['batch'], shuffle=False)

        # Load model
        model = Transformer_base_model.Encoder(vocab_size, params['embed_size'], params['num_layers'], params['heads'], device, params['forward_expansion'], params['dropout'], max_length, params['batch']).to(device)
        checkpoint = torch.load(checkpoint_path)
        #print(checkpoint)
        model.load_state_dict(checkpoint, strict=False)

        # Extract latent vectors and labels
        latent_space, labels_space = [], []
        with torch.no_grad():
            for data_loader in [test_loader, train_loader]:
                for inputs, labels in data_loader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    _, embeddings = model(inputs)
                    latent_space.extend(embeddings.cpu().numpy())
                    labels_space.extend(labels.cpu().numpy())

        # Dimensionality Reduction (PCA and t-SNE)
        latent_space_array = np.array(latent_space)
        labels_space_array = np.array(labels_space).flatten()

        # PCA
        pca = PCA(n_components=2)
        latent_space_pca = pca.fit_transform(latent_space_array)

        # t-SNE
        tsne = TSNE(n_components=2, perplexity=30, n_iter=3000)
        latent_space_tsne = tsne.fit_transform(latent_space_array)

        # Visualization
        plt.figure(figsize=(10, 8))
        cmap = mcolors.ListedColormap(['Cyan', 'Magenta'])
        plt.scatter(latent_space_pca[:, 0], latent_space_pca[:, 1], c=labels_space_array, cmap=cmap, s=50)
        plt.title('PCA of Latent Space')
        plt.xlabel('PC1')
        plt.ylabel('PC2')

        # Save Visualization
        legend_elements = [Line2D([0], [0], marker='o', color='w', label='NonCAS', markersize=10, markerfacecolor='Cyan'),
                   Line2D([0], [0], marker='o', color='w', label='CAS12', markersize=10, markerfacecolor='Magenta')]
        plt.legend(handles=legend_elements, title='Labels')
        save_path = os.path.join(base_folder, f'{folder_name}\\Fold_{fold}', 'latent_space_visualization.png')
        plt.savefig(save_path, bbox_inches='tight')
        plt.close()

        print(f"Saved Latent Space Visualization for {folder_name}, Fold {fold}")

Processing: Cas12_vs_nonCas_labels_0_Cross_Validation_Results, Fold: 1


E:\Anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
E:\Anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


Saved Latent Space Visualization for Cas12_vs_nonCas_labels_0_Cross_Validation_Results, Fold 1
Processing: Cas12_vs_nonCas_labels_0_Cross_Validation_Results, Fold: 2


E:\Anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
E:\Anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


Saved Latent Space Visualization for Cas12_vs_nonCas_labels_0_Cross_Validation_Results, Fold 2
Processing: Cas12_vs_nonCas_labels_0_Cross_Validation_Results, Fold: 3


...

Saved Latent Space Visualization for Cas12_vs_nonCas_labels_0_Cross_Validation_Results, Fold 8
Processing: Cas12_vs_nonCas_labels_0_Cross_Validation_Results, Fold: 9


E:\Anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
E:\Anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


Saved Latent Space Visualization for Cas12_vs_nonCas_labels_0_Cross_Validation_Results, Fold 9
Processing: Cas12_vs_nonCas_labels_0_Cross_Validation_Results, Fold: 10


E:\Anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
E:\Anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


Saved Latent Space Visualization for Cas12_vs_nonCas_labels_0_Cross_Validation_Results, Fold 10


In [ ]:
latent_space_array = np.array(latent_space)
labels_space_array = np.array(labels_space)
#PCA
pca = PCA(n_components=2)  # Reduce to 2 dimensions
latent_space_pca = pca.fit_transform(latent_space_array)

tsne = TSNE(n_components=2, perplexity=30, n_iter=3000)  # Adjust parameters as needed
latent_space_tsne = tsne.fit_transform(latent_space_array)

In [ ]:
plt.figure(figsize=(10, 8))
# Create a custom color map with red and blue
cmap = mcolors.ListedColormap(['Cyan', 'Magenta'])

# For PCA
plt.scatter(latent_space_pca[:, 0], latent_space_pca[:, 1], c=labels_space_array, cmap=cmap, s=50)

# For t-SNE (comment out the above two lines for PCA before using these)
# plt.scatter(latent_space_tsne[:, 0], latent_space_tsne[:, 1], c=labels, cmap='viridis', s=50)

#plt.colorbar()
plt.title('PCA of Latent Space' if 'latent_space_pca' in locals() else 't-SNE of Latent Space')
plt.xlabel('Principal Component 1 (PC1)')
plt.ylabel('Principal Component 2 (PC2)')

# Create custom legend entries
from matplotlib.lines import Line2D
legend_elements = [Line2D([0], [0], marker='o', color='w', label='NonCAS', markersize=10, markerfacecolor='Cyan'),
                   Line2D([0], [0], marker='o', color='w', label='CAS12', markersize=10, markerfacecolor='Magenta')]
plt.legend(handles=legend_elements, title='Labels')
#save_file='latent_space_cas12_nonCas_scratch.png'
#plt.savefig(save_file, bbox_inches='tight')
plt.show()